In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gregorut/videogamesales")

csv_path = path + "/vgsales.csv"

In [2]:
import polars as pl

vgsales = pl.read_csv(csv_path, null_values=["N/A"])
# Show the first 5 rows
print(vgsales.head())


shape: (5, 11)
┌──────┬──────────────────┬──────────┬──────┬───┬──────────┬──────────┬─────────────┬──────────────┐
│ Rank ┆ Name             ┆ Platform ┆ Year ┆ … ┆ EU_Sales ┆ JP_Sales ┆ Other_Sales ┆ Global_Sales │
│ ---  ┆ ---              ┆ ---      ┆ ---  ┆   ┆ ---      ┆ ---      ┆ ---         ┆ ---          │
│ i64  ┆ str              ┆ str      ┆ i64  ┆   ┆ f64      ┆ f64      ┆ f64         ┆ f64          │
╞══════╪══════════════════╪══════════╪══════╪═══╪══════════╪══════════╪═════════════╪══════════════╡
│ 1    ┆ Wii Sports       ┆ Wii      ┆ 2006 ┆ … ┆ 29.02    ┆ 3.77     ┆ 8.46        ┆ 82.74        │
│ 2    ┆ Super Mario      ┆ NES      ┆ 1985 ┆ … ┆ 3.58     ┆ 6.81     ┆ 0.77        ┆ 40.24        │
│      ┆ Bros.            ┆          ┆      ┆   ┆          ┆          ┆             ┆              │
│ 3    ┆ Mario Kart Wii   ┆ Wii      ┆ 2008 ┆ … ┆ 12.88    ┆ 3.79     ┆ 3.31        ┆ 35.82        │
│ 4    ┆ Wii Sports       ┆ Wii      ┆ 2009 ┆ … ┆ 11.01    ┆ 3.28     ┆ 2.96

In [3]:
print(vgsales.columns)
print(len(vgsales['Name'].unique()))
print(vgsales.shape)
print(vgsales.dtypes)

vgsales.drop('Rank')
vgsales.drop('Global_Sales')



['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
11493
(16598, 11)
[Int64, String, String, Int64, String, String, Float64, Float64, Float64, Float64, Float64]


Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
i64,str,str,i64,str,str,f64,f64,f64,f64
1,"""Wii Sports""","""Wii""",2006,"""Sports""","""Nintendo""",41.49,29.02,3.77,8.46
2,"""Super Mario Bros.""","""NES""",1985,"""Platform""","""Nintendo""",29.08,3.58,6.81,0.77
3,"""Mario Kart Wii""","""Wii""",2008,"""Racing""","""Nintendo""",15.85,12.88,3.79,3.31
4,"""Wii Sports Resort""","""Wii""",2009,"""Sports""","""Nintendo""",15.75,11.01,3.28,2.96
5,"""Pokemon Red/Pokemon Blue""","""GB""",1996,"""Role-Playing""","""Nintendo""",11.27,8.89,10.22,1.0
…,…,…,…,…,…,…,…,…,…
16596,"""Woody Woodpecker in Crazy Cast…","""GBA""",2002,"""Platform""","""Kemco""",0.01,0.0,0.0,0.0
16597,"""Men in Black II: Alien Escape""","""GC""",2003,"""Shooter""","""Infogrames""",0.01,0.0,0.0,0.0
16598,"""SCORE International Baja 1000:…","""PS2""",2008,"""Racing""","""Activision""",0.0,0.0,0.0,0.0


In [4]:
print([x + ", " + str(vgsales[x].is_null().sum()) for x in vgsales.columns])


['Rank, 0', 'Name, 0', 'Platform, 0', 'Year, 271', 'Genre, 0', 'Publisher, 58', 'NA_Sales, 0', 'EU_Sales, 0', 'JP_Sales, 0', 'Other_Sales, 0', 'Global_Sales, 0']


In [5]:
vgsales = vgsales.drop_nulls()

print(vgsales.columns)
print(vgsales.shape)
print(vgsales.dtypes)
print([x + ", " + str(vgsales[x].is_null().sum()) for x in vgsales.columns])


['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
(16291, 11)
[Int64, String, String, Int64, String, String, Float64, Float64, Float64, Float64, Float64]
['Rank, 0', 'Name, 0', 'Platform, 0', 'Year, 0', 'Genre, 0', 'Publisher, 0', 'NA_Sales, 0', 'EU_Sales, 0', 'JP_Sales, 0', 'Other_Sales, 0', 'Global_Sales, 0']


In [6]:
# load vào clickhouse

from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()

# Lấy user và password từ environment
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")


from clickhouse_driver import Client
client = Client(
    host='localhost',
    port=9000,
    user=db_user,
    password=db_password
)


client.execute(
    'INSERT INTO vgsales_raw.vgsales_raw (Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales) VALUES',
    vgsales.to_dicts()  # Chuyển DataFrame Polars thành list of dicts
)

16291